Jupyter Notebook to load and plot the parcels simulation output

In [ ]:
#load packages
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from netCDF4 import Dataset
import netCDF4 as nc4
import numpy as np
from cartopy import config
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import math
import scipy.io
from scipy.io import loadmat
from scipy.stats import binned_statistic_2d


# Set some plotting defaults
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 100
#read in land feature for later
land_110m = cfeature.NaturalEarthFeature('physical', 'land', '110m',
                                        edgecolor='face',
                                        facecolor=cfeature.COLORS['land'])

# Set general font size
plt.rcParams['font.size'] = '16' 

Read in and print the variables in the netcdf file - you will need to amend the directory location to your home directory and the outfile name that you used in the parcels_run.py script

In [ ]:
outfile=nc4.Dataset('/home/users/trainXXX/lagran/test.nc','r',format='NETCDF4')
print(outfile)

Read in the variables in the file

.T transposes the shape of the variable which makes plotting easier

In [ ]:
lat=outfile.variables['lat'][:,:].T #latitude in degrees 
lon=outfile.variables['lon'][:,:].T #longitude in degrees
depth=outfile.variables['z'][:,:].T #depth in meters
time=outfile.variables['time'][:,:].T #time in seconds
bathy=outfile.variables['bathy'][:,:].T #bathymetry depth in meters
mld=outfile.variables['mld'][:,:].T #along trajectory mixed layer depth in meters
obs=outfile.variables['obs'][:].T #number of outputted timesteps
traj=outfile.variables['trajectory'][:].T #trajectory number

Explore what the different variables look like by changing the var=XXX - if you pick a one dimensional variable you will need to change var[:,:] to var[:]

In [ ]:
var=depth #change variable name here
print(var[2,0:100])
print(var.shape) #shape of the variables

var=mld #change variable name here
print(var[0:5,0:20])
print(var.shape) #shape of the variables

Let's convert the time variable into a more useful format as it's currently in seconds

In [ ]:
time_days=time.copy()/86400 #number of seconds in a day # converting to days from start of simulation
print(time_days[:,0])
#but we have also divided the fill value of -9.22337204e+18 by 86400 - let's get rid of this
time_days[time_days==-106751991167300.64]=np.nan #replace with NaNs
print(time_days[:,0])

Plot the trajectory release grid

In [ ]:
plt.figure()
plt.scatter(lon[0,:],lat[0,:])
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()
plt.close()

In [ ]:
#the positive and negative longitude values are separated to avoid stripes along the plot if a trajectory crosses -180 to 180 or vice versa
lon_180_0=lon.copy()
lon_180_0[lon>0]=np.nan
lon_0_180=lon.copy()
lon_0_180[lon<0]=np.nan

plt.figure()
plt.plot(lon_180_0,lat,zorder=0) # plotting trajectory lon and lats
plt.plot(lon_0_180,lat,zorder=0) # plotting trajectory lon and lats
plt.scatter(lon[0,:],lat[0,:],c='k',s=0.9)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()
plt.close()

Let's plot a map of trajectories - adjust the lon and lat range to zoom in to the correct area

In [ ]:
xmin=-180 #set minimum longitude limit
xmax=180 #set maximum longitude limit
ymin=-90 #set minimum latitude limit
ymax=90 #set maximum longitude limit
xint=30 #set xtick interval
yint=30 #set ytick interval



plt.figure()
ax1=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
plot1 = ax1.plot(lon_180_0,lat,transform=ccrs.PlateCarree(),zorder=-1) # plotting trajectory lon and lats
plot2 = ax1.plot(lon_0_180,lat,transform=ccrs.PlateCarree(),zorder=-1) # plotting trajectory lon and lats
ax1.scatter(lon[0,:],lat[0,:],c='k',s=0.4)
ax1.add_feature(land_110m,facecolor='gray')
ax1.coastlines(resolution='110m')
ax1.set_ylim([ymin,ymax])
ax1.set_xlim([xmin,xmax])
ax1.set_xticks(np.arange(xmin,xmax+0.5, step=xint))
ax1.set_yticks(np.arange(ymin, ymax+0.5, step=yint))
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
plt.show()
plt.close()

Plot showing how the depth of the trajectories change with time 

In [ ]:
plt.figure()
plt.plot(time_days,depth)
plt.gca().invert_yaxis()
plt.xlabel('Time (days from start of simulation)')
plt.ylabel('Depth (m)')
plt.show()
plt.close()

Plot a heatmaps of the trajectory density - the warmer the colour the more trajectory timesteps that passed through that grid cell

In [ ]:
#create meshgrid and arrange data
xgrid=np.arange(-180,180.25,1) #change the final number to indicate grid resolution, 1=1 degree
ygrid=np.arange(-90,90.25,1) #change the final number to indicate grid resolution, 1=1 degree

#grid lon and lat and bin the data
X, Y= np.meshgrid(xgrid,ygrid)
histgrid, xgrid, ygrid= np.histogram2d(lon.flatten(),lat.flatten(), bins=(xgrid, ygrid))

#plot figure
plt.figure()
ax1=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
axins1 = inset_axes(ax1,
                   width="5%",  # width = 5% of parent_bbox width
                   height="100%",  # height : 50%
                   loc='lower left',
                   bbox_to_anchor=(1.05, 0., 1, 1),
                   bbox_transform=ax1.transAxes,
                   borderpad=0)
cmap=cm.get_cmap("plasma",lut=24)
plot1=ax1.pcolormesh(X,Y,histgrid.T,cmap=cmap,vmin=1,transform=ccrs.PlateCarree())
plot1.cmap.set_under('w')
ax1.add_feature(land_110m,facecolor='gray')
ax1.coastlines(resolution='110m')
cb1=plt.colorbar(plot1,cax=axins1)
cb1.set_label('Trajectory Density')
ax1.set_ylim([ymin,ymax])
ax1.set_xlim([xmin,xmax])
ax1.set_xticks(np.arange(xmin,xmax+0.5, step=xint))
ax1.set_yticks(np.arange(ymin, ymax+0.5, step=yint))
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
plt.show()
plt.close()



Sequestration Efficiency Calculations similar to Baker et al. (2022)

In [ ]:
#create variable to indicate where the trajectory is within the MLD
reentrain=depth.copy()
reentrain[0,:]=np.nan
reentrain[depth>mld]=0 
reentrain[depth<mld]=1

#indicate which trajectories were reentrained into the mixed layer
reen_traj=np.nansum(reentrain,axis=0)
reen_traj[reen_traj>=1]=1

#calculate the sequestration efficiency of the simulation
seqeff=100-(np.nansum(reen_traj)/len(depth[0,:])*100)

#change YY to depth of particle release and change XX to length of simulation in years
print('Sequestration Efficiency (%) at YY m over timescales of XX years')
print(np.round(seqeff,2))

Let's check that the calculated sequestration efficiency seems correct but looking at the data - all dark blue plots == 100% sequestration efficiency, in plot b) all yellow markers at one timestep == 0% sequestration efficiency

In [ ]:
plt.figure()
plt.scatter(depth,mld,c=reentrain,s=3)
plt.xlim([0,900])
plt.ylim([0,900]) #you may need to expand these limits if the plot is empty
plt.colorbar()
plt.xlabel('Depth (m)')
plt.ylabel('MLD (m)')
plt.title('Yellow markers = trajectory timesteps in the mixed layer',fontsize=12)
plt.show()
plt.close()

plt.figure()
plt.scatter(time_days, depth,c=reentrain,s=1)
plt.gca().invert_yaxis()
plt.colorbar()
plt.xlabel('Time (days from start of simulation')
plt.ylabel('Depth(m)')
plt.show()
plt.close()

Plot a map of the spatial sequestration efficiency at the release location

In [ ]:
#some calculations to get the gridded sequestration efficiency
reen_grid=reentrain.copy()
reen_lat=np.full([len(reen_grid[:,0]),len(reen_grid[0,:])],np.nan)
reen_lon=np.full([len(reen_grid[:,0]),len(reen_grid[0,:])],np.nan)

for i in range(0,len(reen_grid[0,:]),1):
    reen_first=0
    for j in range(0,len(reen_grid[:,0]),1):
        if reen_first==1 and reen_grid[j,i]==1:
            reen_grid[j,i]=0
            
        elif reen_first==0 and reen_grid[j,i]==1:
            reen_first=1
            reen_lat[j,i]=lat[0,i]
            reen_lon[j,i]=lon[0,i]
            
            
rel_lon=lon[0,:]
rel_lat=lat[0,:]
rel_traj=depth[0,:]
rel_traj[rel_traj>0]=1

xi = np.arange(-180,180.25,1)
yi = np.arange(-90,90.25,1)
#calculate how many retrained 
reen_sum=scipy.stats.binned_statistic_2d(reen_lon.flatten(), reen_lat.flatten(), reen_grid.flatten(), statistic='sum', bins=[xi,yi], range=None, expand_binnumbers=False).statistic
print('Number of trajectories reentrained into the mixed layer')
print(np.nansum(reen_sum))

rel_sum=scipy.stats.binned_statistic_2d(rel_lon.flatten(), rel_lat.flatten(), rel_traj.flatten(), statistic='sum', bins=[xi,yi], range=None, expand_binnumbers=False).statistic
print('Number of trajectories in the simulation')
print(np.nansum(rel_sum))

#calculate the gridded sequestration efficiency
seqeff_grid=100-((reen_sum/rel_sum)*100)

In [ ]:
levls=np.linspace(0,100,11)
#plot figure
plt.figure()
ax1=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
axins1 = inset_axes(ax1,
                   width="5%",  # width = 5% of parent_bbox width
                   height="100%",  # height : 50%
                   loc='lower left',
                   bbox_to_anchor=(1.05, 0., 1, 1),
                   bbox_transform=ax1.transAxes,
                   borderpad=0)
cmap=cm.get_cmap("plasma",lut=20)
plot1=ax1.pcolormesh(xi[0:len(xi)-1],yi[0:len(yi)-1],seqeff_grid.T,vmin=0,vmax=100,cmap=cmap,transform=ccrs.PlateCarree())
plot1.cmap.set_under('w')
ax1.add_feature(land_110m,facecolor='gray')
ax1.coastlines(resolution='110m')
cb1=plt.colorbar(plot1,cax=axins1,ticks=levls)
cb1.set_label('Sequestration Efficiency (%)')
ax1.set_ylim([ymin,ymax])
ax1.set_xlim([xmin,xmax])
ax1.set_xticks(np.arange(xmin,xmax+0.5, step=xint))
ax1.set_yticks(np.arange(ymin, ymax+0.5, step=yint))
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
plt.show()
plt.close()

Now switch back to the task sheet to run the trajectory_metrics.py script to calculate distances and speed

Keep exploring the simulation results by creating your own plots below... have fun!  

In [ ]:
#make more plots here...